#Install Dependencies

!! Use GPU runtime

In [ ]:
!pip install -qr requirements.txt  # install dependencies
import torch

from IPython.display import Image, clear_output 

# clear_output()
print(torch.__version__)
print(torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.5 MB/s eta 0:00:00
Setup complete. Using torch 2.0.0+cu118 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15101MB, multi_processor_count=40)


# Define Model Configuration and Architecture

Write a yaml script that defines paths for train and validation datasets

# yaml file for training
<p></p>
train: path to train dataset</p>
val: path to validation dataset</p>

nc: 4</p>

# Train Custom YOLOv5 Detector


Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** set the path to our yaml file
- **cfg:** specify our model configuration
- **weights:** specify a custom path to weights. 
- **name:** result names
- **nosave:** only save the final checkpoint
- **cache:** cache images for faster training

In [ ]:
# time its performance
"""
Parameters for this case:
image size = 416
batch = 16
epochs = 200
dataset info = /content/yolov5/data.yaml
model configuration = yoloS
output directory = yolo5s results
"""
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 200 --data /content/yolov5/data.yaml --cfg /content/yolov5/models/yolo5s.yaml --weights '' --name yolov5s_results  --cache

# Evaluation

In [ ]:
# Start tensorboard

%load_ext tensorboard
%tensorboard --logdir /content/yolov5/runs/

In [ ]:
%cd /content/yolov5/
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/content/yolov5/runs/train/yolov5s_results/results.png', width=1000)  # view results.png

In [ ]:
# print out an augmented training example
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results/train_batch0.jpg', width=900)

#Run Inference

In [ ]:
# when we ran this, we saw .007 second inference time. That is 140 FPS on a TESLA P100!
# use the best weights!
!rm -rf /content/run/
%mkdir /content/run
%cd /content/yolov5/
#inputs
# weight file
# test images
!python detect.py --weights /content/best.pt --img 416 --conf 0.4 --source /content/part/

In [42]:
# to write videos
import cv2
import os

# Define the folder containing the images
folder_path = "/content/run/"

# Define the output video file name
output_video = "output.avi"

# Get the list of image files in the folder
image_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.jpg') or f.endswith('.png')])

# Get the first image in the list to use as a reference for the video dimensions
img = cv2.imread(image_files[0])
height, width, layers = img.shape

# Initialize the video writer
video = cv2.VideoWriter(output_video, 0, 1, (width, height))

# Loop through the image files and add each frame to the video
for image_file in image_files:
    img = cv2.imread(image_file)
    video.write(img)

# Release the video writer and destroy all windows
video.release()
cv2.destroyAllWindows()